Modeling Notebook:

# Loading Packages

In [1]:
import pandas as pd
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Loading Data

In [2]:
# Define the path to your CSV file
# file_path = r"C:/Users/johne/Downloads/IWC_Work_Orders_Extract.csv"
file_path = r"C:/Users/matt/Downloads/IWC_Work_Orders_Extract.csv"
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

C:\Users\matt\AppData\Local\Temp\ipykernel_6572\608475065.py:5: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


# Glypse of Data

In [3]:
# Display the first 5 rows of the DataFrame
print("Head of the DataFrame:")
print(df.head())

Head of the DataFrame:
    ORDER_ID PLANT_ID PRODUCTION_LOCATION EXECUTION_START_DATE  \
0  705642457     G812                ROMA           2024-05-04   
1  704191697     G812                ROMA           2022-09-13   
2  704466547     G812                ROMA           2022-12-21   
3  703834477     G812                ROMA           2022-07-04   
4  704661125     G291               MONZA           2023-03-15   

  EXECUTION_FINISH_DATE ACTUAL_START_TIME ACTUAL_FINISH_TIME  \
0            2024-05-12      06:00:00.000       23:04:08.000   
1            2022-09-13      06:00:00.000       17:17:24.000   
2            2022-12-21      07:00:00.000       07:00:00.000   
3            2022-07-04      06:00:00.000       06:00:00.000   
4            2023-03-15      07:00:00.000       07:00:00.000   

   ACTUAL_WORK_IN_MINUTES MAINTENANCE_PLAN  MAINTENANCE_ITEM  ...  \
0                   390.0              NaN               NaN  ...   
1                   420.0              NaN               

# Creating new variables:

In [4]:
# Calculating Time Between Maintenance Events Section
# Convert 'EXECUTION_START_DATE' to datetime format
df['EXECUTION_START_DATE'] = pd.to_datetime(df['EXECUTION_START_DATE'], errors='coerce')

# Drop rows with invalid 'EXECUTION_START_DATE'
df.dropna(subset=['EXECUTION_START_DATE'], inplace=True)

# Sort the DataFrame by 'EQUIPMENT_ID' and 'EXECUTION_START_DATE'
df.sort_values(by=['EQUIPMENT_ID', 'EXECUTION_START_DATE'], inplace=True)

# Calculate the time difference between consecutive 'EXECUTION_START_DATE' for each 'EQUIPMENT_ID'
df['TIME_BETWEEN_MAINTENANCE'] = df.groupby('EQUIPMENT_ID')['EXECUTION_START_DATE'].diff().dt.days

# Display the first 5 rows to verify the new column
print("Head of the DataFrame with Time Between Maintenance:")
print(df.head())

Head of the DataFrame with Time Between Maintenance:
          ORDER_ID PLANT_ID PRODUCTION_LOCATION EXECUTION_START_DATE  \
224341   702834897     G816                COTA           2021-03-30   
1021363  702233571     G812                ROMA           2020-05-30   
449320   702376206     G812                ROMA           2020-08-04   
712483   702477379     G812                ROMA           2020-09-21   
844336   702643176     G812                ROMA           2020-12-09   

        EXECUTION_FINISH_DATE ACTUAL_START_TIME ACTUAL_FINISH_TIME  \
224341             2021-03-30      06:00:00.000       06:00:00.000   
1021363            2020-05-30      06:00:00.000       06:00:00.000   
449320             2020-08-04      06:00:00.000       06:00:00.000   
712483             2020-09-21      06:00:00.000       06:00:00.000   
844336             2020-12-09      07:00:00.000       07:00:00.000   

         ACTUAL_WORK_IN_MINUTES MAINTENANCE_PLAN  MAINTENANCE_ITEM  ...  \
224341            

# Model

In [ ]:
# Model
# Predictive Model Section
# Drop rows with missing 'TIME_BETWEEN_MAINTENANCE' (since the first entry for each equipment will have NaN)
df.dropna(subset=['TIME_BETWEEN_MAINTENANCE'], inplace=True)

# Selecting features for the predictive model
# We will use 'TIME_BETWEEN_MAINTENANCE' and other relevant features to predict the next maintenance event
features = ['TIME_BETWEEN_MAINTENANCE', 'EQUIPMENT_ID', 'ACTUAL_WORK_IN_MINUTES']
X = df[features]
y = df['TIME_BETWEEN_MAINTENANCE'].shift(-1)  # Target is the next 'TIME_BETWEEN_MAINTENANCE' value

# Drop the last row since it has NaN target value
y.dropna(inplace=True)
X = X.iloc[:-1, :]

# Encoding categorical features ('EQUIPMENT_ID')
X = pd.get_dummies(X, columns=['EQUIPMENT_ID'])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling Features Section
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training a Predictive Model to Estimate Maintenance Time Section
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Model Evaluation Section
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for Predictive Model: {mse}')

# Average Maintenance Time Prediction per Equipment ID Section
avg_maintenance_time = df.groupby('EQUIPMENT_ID')['TIME_BETWEEN_MAINTENANCE'].mean().reset_index()

# Visualization Section
plt.figure(figsize=(12, 6))
sns.barplot(x='EQUIPMENT_ID', y='TIME_BETWEEN_MAINTENANCE', data=avg_maintenance_time)
plt.xlabel('Equipment ID')
plt.ylabel('Average Time Between Maintenance (Days)')
plt.title('Average Maintenance Time per Equipment ID')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Glypse of Data Section
# Display the first 5 rows of the DataFrame
print("Head of the DataFrame:")
print(df.head())